In [1]:
from tqdm import tqdm
import os
import json
import random

In [2]:
apikey = ''

In [3]:
import requests

def curl(value):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {apikey}',
    }

    json_data = {
        'messages': [
            {'role': 'system', 'content':'answer only, no need to explain'},
            {
                'role': 'user',
                'content': value,
            },
        ],
        'max_tokens': 100,
        'temperature': 0,
        'model': 'gpt-4-1106-preview',
    }

    response = requests.post(
        'https://api.openai.com/v1/chat/completions',
        headers=headers,
        json=json_data,
        timeout = 120,
    )
    
    return response

In [4]:
with open('BM-A-pt3') as fopen:
    text = fopen.read()
    
questions = []
for t in text.split('no: ')[1:]:
    t = t.strip()
    no = t.split('\n')[0]
    objektif = t.split('objektif: ')[1].split('\n')[0]
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'objektif': objektif,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    questions.append(data)

In [5]:
arange = set(range(len(questions)))

In [6]:
def convert_prompt(row, answer = False):
    if answer:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
    """
    return prompt.strip()

In [7]:
i = 0
shots = random.sample(arange - {i}, 3)
prompts = []
for no, s in enumerate(shots):
    prompts.append(f'Contoh soalan {no + 1}\n' + convert_prompt(questions[s], answer = True))

prompts.append(convert_prompt(questions[i]))
prompt = '\n\n'.join(prompts)
print(prompt)

Contoh soalan 1
objektif: Baca petikan di bawah ini dengan teliti, kemudian jawab soalan-soalan yang berikut.
soalan: Maka diambil padi yang dijemur itu segenggam, lalu disembunyikan butir-butir padi itu di
dalam lubang-lubang luka pada tumitnya. Setelah itu ia pun meminta izin dengan baik-
baik untuk pulang ke kampungnya, ke bumi tempat saudaranya yang tetap menantinya.
Maka berkatalah induk semangnya, orang kayangan yang masih juga mencurigainya
itu, "Baiklah jikalau kamu ingin pulang ke kampung kamu, akan tetapi, sebelum itu saya
akan memeriksa dan menggeledah kamu lebih dahulu. Barangkali ada padi yang
disembunyikan untuk kamu bawa sebagai oleh-oleh ke bumi."
Anak yatim piatu itu pun menjawab dengan tenangnya, "Silakan ! Periksalah dengan
teliti kalau-kalau ada apa-apa yang saya bawa. Sekarang saya sudah betul-betul jera
mengambil apa-apa. Saya sungguh-sungguh sudah bertaubat dan tidak mahu lagi
melakukannya."
Kemudian induk semang berkata, "Jikalau benar demikian katamu, maka jela

In [8]:
r = curl(prompt)

In [9]:
r.json()['choices'][0]['message']['content']

'jawapan: D'

In [10]:
for i in tqdm(range(len(questions))):
    shots = random.sample(arange - {i}, 3)
    prompts = []
    for no, s in enumerate(shots):
        prompts.append(f'Contoh soalan {no + 1}\n' + convert_prompt(questions[s], answer = True))

    prompts.append(convert_prompt(questions[i]))
    prompt = '\n\n'.join(prompts)
    try:
        r = curl(prompt)
        answer = r.json()['choices'][0]['message']['content']
        questions[i]['output'] = answer
    except:
        pass

100%|███████████████████████████████████████████| 54/54 [00:45<00:00,  1.18it/s]


In [11]:
import json

with open('output-3shot.json', 'w') as fopen:
    json.dump(questions, fopen)

In [12]:
filtered = [q for q in questions if 'output' in q]
len(filtered)

54

In [13]:
correct = 0
for q in filtered:
    correct += q['output'].replace('jawapan:', '').split('.')[0].strip() == q['jawapan']
(correct / len(filtered)) * 100

55.55555555555556